<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [30]:
import sys
import time
import os
import json 
import pandas as pd
import random

from os.path import dirname, abspath
root_path = 'C:\Repos\HUJI\snpnmi'
sys.path.append(root_path)

from utils.common import normalize_distances, get_number_of_windows_by_class, build_empty_upper_left_matrix, write_upper_left_matrix_to_file, str2bool, get_paths_helper
from utils.config import get_num_chrs, get_num_individuals

random.seed(a='42', version=2)

# params
mac=-1
maf=0.49


print('mac',mac)
print('maf',maf)
mac_maf = 'maf'
class_name = '0.49'
#  we dont want to use the same seed here, as we will have the same values for all clasees
paths_helper = get_paths_helper()

# inputs are /vol/sci/bio/data/gil.greenbaum/amir.rubin/vcf/hgdp/classes/chr21/mac_2.012
input_per_chr_template = paths_helper.classes_folder + 'chr{chr_id}/' + f'{mac_maf}_{class_name}.012'

# outputs are 
# indexes: /vol/sci/bio/data/gil.greenbaum/amir.rubin/vcf/hgdp/classes/windows/indexes/windows_indexes_for_class_2.json
windows_indexes_file = paths_helper.windows_indexes_template.format(class_name=class_name)

# transposed windows: 
# /vol/sci/bio/data/gil.greenbaum/amir.rubin/vcf/hgdp/classes/windows/mac_2/count_dist_window_0_transposed.tsv.gz
windows_transposed_template = paths_helper.windows_folder + f'{mac_maf}_{class_name}/count_dist_window_' + '{window_id}_transposed.tsv.gz'

print(input_per_chr_template)
print(windows_indexes_file)
print(windows_transposed_template)

# make sure output folders exists
os.makedirs(paths_helper.windows_indexes_folder, exist_ok=True)
os.makedirs('/'.join(windows_transposed_template.split('/')[:-1]), exist_ok=True)
number_of_windows = get_number_of_windows_by_class()[str(class_name)]
print(number_of_windows)

num_chrs = get_num_chrs()
print(num_chrs)

# go over chrs
for chr_id in range(1,num_chrs+1):
    s = time.time()
    print(f'{s} Start process chr {chr_id}')
    chr_class_file = input_per_chr_template.format(chr_id=chr_id)
    random_split_class_from_chr_to_windows(chr_class_file, number_of_windows, windows_indexes_file, windows_transposed_template)


print(f'{(time.time()-s)/60} minutes total run time')

mac -1
maf 0.49
C:/Data/HUJI/vcf/hgdp/classes/chr{chr_id}/maf_0.49.012
C:/Data/HUJI/vcf/hgdp/classes/windows/indexes/windows_indexes_for_class_0.49.json
C:/Data/HUJI/vcf/hgdp/classes/windows/maf_0.49/count_dist_window_{window_id}_transposed.tsv.gz
988
22
1614762218.1741512 Start process chr 1
1614762219.242577 Start process chr 2
1614762220.404365 Start process chr 3
1614762221.4092982 Start process chr 4
1614762222.415291 Start process chr 5


KeyboardInterrupt: 

In [26]:
def append_to_dic(dic, key, value):
    if not key in dic.keys():
        dic[key] = []
    dic[key].append(value)

In [39]:
import gzip
#def random_split_class_from_chr_to_windows(chr_id, chr_class_file, number_of_windows, windows_indexes_file, windows_transposed_template):
# load file to memory
with open(chr_class_file,'r') as f:
        num_columns = len(f.readline().split('\t'))
names = [f'idx{i}' for i in range(num_columns)]
df_012 = pd.read_csv(chr_class_file, sep='\t', names= names)
print(f'Number of sites: {len(df_012.columns)}')
num_ind = len(df_012)
assert num_ind == get_num_individuals()

window_index_2_site_index = dict()
i = 0
for (columnName, columnData) in df_012.iteritems():
    i += 1
    if i%1000 == 0:
        print(f'Done {i}/{len(df_012.columns)} sites' )
    window_index = random.randint(1, number_of_windows)
    # store the index in the file mapping windows to indexes used
    append_to_dic(window_index_2_site_index, window_index, f'{chr_id};{columnName[3:]}')
    #print(f'write {columnName} to {window_index}')    
    # append the column as a row to the right window file
    window_transposed_file = windows_transposed_template.format(window_id=window_index)
    with gzip.open(window_transposed_file,'ab') as f:
        s = '\t'.join([str(i) for i in columnData.values]) + '\n'
        f.write(s.encode()) 

with open(windows_indexes_file, "w" ) as f: 
        json.dump(window_index_2_site_index, f )

In [46]:
# transpose and output to file
with gzip.open(window_transposed_file,'rb') as f:
        num_columns = len(f.readline().decode().split('\t'))
names = [f'{i+1}' for i in range(num_columns)]
df1 = pd.read_csv(window_transposed_file, compression='gzip', sep='\t', names=names) 
df1.transpose()
#TODO window_not_transposed_file
df1.transpose().to_csv(window_not_transposed_file,index=True,header=False, compression='gzip')


929

In [52]:
window_transposed_file

'C:/Data/HUJI/vcf/hgdp/classes/windows/maf_0.49/count_dist_window_946_transposed.tsv.gz'